In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1223: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [85]:
kinogo = pd.read_csv('kinogo.csv')

In [86]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    #print(model_output)
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

kinogo['description'] = kinogo['description'].map(lambda text: embed_bert_cls(text, model, tokenizer))

In [87]:
kinogo.head(10)

,page_url,image_url,movie_title,description
0,https://kinogo.online/filmy/102112-hanna-v-igr...,kinogo.online/uploads/mini/fullstory/75/d88deb...,Ханна. В игре (2024),"[0.06898929, 0.056686744, 0.04241453, -0.02848..."
1,https://kinogo.online/filmy/102111-bajkery.html,kinogo.online/uploads/mini/fullstory/f5/fdcada...,Байкеры (2023),"[0.09805015, -0.009207892, 0.020433972, 0.0045..."
2,https://kinogo.online/filmy/102021-jekzorcizm....,kinogo.online/uploads/mini/fullstory/f3/f419ff...,Экзорцизм (2024),"[0.023994686, 0.03418008, 0.0059999814, -0.019..."
3,https://kinogo.online/filmy/102142-bolero-dush...,kinogo.online/uploads/mini/fullstory/30/43d33a...,Болеро. Душа Парижа (2024),"[0.08660622, 0.03467266, 0.031870443, 0.011732..."
4,https://kinogo.online/filmy/102271-zombi-v-seu...,kinogo.online/uploads/mini/fullstory/1c/c07ec9...,Зомби в Сеуле (2024),"[0.08400867, 0.028651478, 0.023705168, -0.0121..."
5,https://kinogo.online/filmy/54528-kto-ugodno-k...,kinogo.online/uploads/mini/fullstory/4e/894333...,"Кто угодно, кроме тебя (2023)","[0.09322013, 0.046440545, 0.028097149, -0.0126..."
6,https://kinogo.online/filmy/51097-kniga-reshen...,kinogo.online/uploads/mini/fullstory/8d/36364f...,Книга решений (2023),"[0.043360803, 0.03410499, 0.010053297, -0.0103..."
7,https://kinogo.online/filmy/1970-vmesto-nas-dv...,kinogo.online/uploads/mini/fullstory/a5/fd307d...,Вместо нас двоих (2016),"[0.07571638, 0.01999731, 0.015330197, -0.00573..."
8,https://kinogo.online/filmy/100588-nuzhdy-pute...,kinogo.online/uploads/mini/fullstory/a0/e268da...,Нужды путешественника (2024),"[0.12101997, 0.038298316, 0.030036567, 0.01013..."
9,https://kinogo.online/filmy/102188-svezhie-ubi...,kinogo.online/uploads/mini/fullstory/d8/37d0ae...,Свежие убийства (2023),"[0.07419464, 0.014064946, 0.040318046, -0.0103..."


In [ ]:
# Фильм про любовь, где богатый мужчина влюбляется в простую женщину и хочет, чтобы она за него вышла замуж, а она любит бомжа
# Фильм про мужчину, который благодаря сверхествестенным силам стал крутым, спас мир, нашел любовь и признание окружающих

In [88]:
search_phrase = 'Комик Артур Флек живет с больной матерью, которая с детства учит его «ходить с улыбкой». Пытаясь нести в мир хорошее и дарить людям радость, Артур сталкивается с человеческой жестокостью и постепенно приходит к выводу, что этот мир получит от него не добрую улыбку,'
search_e = embed_bert_cls(search_phrase, model, tokenizer)

In [89]:
kinogo['sim'] = kinogo.apply(lambda row: cosine_similarity(row['description'].reshape(1,-1),search_e.reshape(1,-1))[0][0], axis=1)

In [90]:
kinogo.sort_values(by=['sim'],ascending=False).head(15)

,page_url,image_url,movie_title,description,sim
5461,https://metarankings.ru/film-paren-s-togo-svet...,https://metarankings.ru/images/uploads/2023/05...,Парень с того света,"[0.03898963, 0.05158177, -0.008744004, -0.0140...",0.931482
5576,https://metarankings.ru/film-paren-s-togo-svet...,https://metarankings.ru/images/uploads/2023/05...,Парень с того света,"[0.03898963, 0.05158177, -0.008744004, -0.0140...",0.931482
5117,https://metarankings.ru/lico-2018/,https://metarankings.ru/images/uploads/Twarz-c...,Лицо,"[0.044134345, 0.017419439, 0.0032808222, -0.03...",0.930449
5350,https://metarankings.ru/liza-2023/,https://metarankings.ru/images/uploads/2023/02...,Лиза,"[0.02541809, 0.025028022, 0.0063415794, -0.051...",0.926997
3005,https://metarankings.ru/dzhoker-2019/,https://metarankings.ru/images/uploads/dzhoker...,Джокер,"[0.018756784, 0.014185255, 0.026707422, -0.029...",0.926481
2603,https://kinogo.online/filmy/49754-riki-dzhervj...,kinogo.online/uploads/mini/fullstory/6e/971f4a...,Рики Джервэйс: Сверхъестественное (2022),"[0.04589148, 0.045875635, 0.021412909, -0.0343...",0.924161
1216,https://kinogo.online/filmy/51571-manodrom.html,kinogo.online/uploads/mini/fullstory/47/74c9fa...,Манодром (2023),"[0.046160728, 0.038198873, 0.015744817, -0.012...",0.919017
5380,https://metarankings.ru/bitva-2019/,https://metarankings.ru/images/uploads/bitva-c...,Битва,"[0.03232536, 0.03516825, -0.009130442, -0.0309...",0.917156
4439,https://metarankings.ru/god-rozhdeniya-2023/,https://metarankings.ru/images/uploads/2023/11...,Год рождения,"[0.04085525, 0.062683485, 0.0034607325, -0.004...",0.916854
4317,https://metarankings.ru/lyudi-iks-tyomnyj-feniks/,https://metarankings.ru/images/uploads/Dark-Ph...,Люди Икс: Тёмный Феникс,"[0.047479436, 0.04970036, 0.033443872, -0.0266...",0.916557
